# CNN development
## Data retrieval

Mounting the data drive.

In [1]:
!apt-get install unrar
!pip install split_folders
!pip install --upgrade pip
!pip install --upgrade Pillow
!pip install tensorflow_datasets
!pip install -U scikit-learn

'''
!mkdir "content"
!mkdir "content/sample_data"
!unrar x "augmented/dataset.rar" -d "content/sample_data"
!mkdir "content/simple_data"
!unrar x "simple/dataset.rar" -d "content/simple_data"
'''

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.5.8-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


'\n!mkdir "content"\n!mkdir "content/sample_data"\n!unrar x "augmented/dataset.rar" -d "content/sample_data"\n!mkdir "content/simple_data"\n!unrar x "simple/dataset.rar" -d "content/simple_data"\n'

Splitting dataset into training, validation, and test subsets.

In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import splitfolders as sf
from tensorflow.compat.v1 import ConfigProto, InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

img_size = 299

val_split = 0.15

data_dir = pathlib.Path("content/sample_data/split/train")
batch_size = 1
img_height = img_size
img_width = img_size

#sf.ratio("content/sample_data", output="content/sample_data/split", seed=123, ratio=(.8, .2))

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=val_split,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=val_split,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  pathlib.Path("content/sample_data/split/val"),
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1744 files belonging to 3 classes.
Using 1483 files for training.
Found 1744 files belonging to 3 classes.
Using 261 files for validation.
Found 437 files belonging to 3 classes.


Acquiring training labels and images from dataset.

In [3]:
import numpy as np

train_y = []
train_X = []

for images, labels in train_ds:
    train_X.append(images.numpy()[0])
    train_y.append(labels.numpy()[0])
    
train_y = np.array(train_y)
train_X = np.array(train_X)

## Auxiliar functions

Parsing hyperparametres to String.

In [4]:
def hyperToString(hyper):
  cad = "model"
  i = 0
  for item in hyper:
    tmp = "_P"+str(i)+"-"+str(item)
    cad += tmp

  return cad

Cross-validation by K-folds.

In [5]:
from sklearn.model_selection import KFold, StratifiedKFold

def KFcrossVal(cnn,hyper,y_train,X_train):
  nSplits = 10
  randomState = 0
  shuffle = True

  print("-------",hyperToString(hyper))

  skf = StratifiedKFold(n_splits = nSplits, random_state = randomState, shuffle = shuffle)

  VALIDATION_ACCURACY = []
  VALIDATION_LOSS = []

  for train_index, val_index in skf.split(np.zeros(len(y_train)),y_train):
    trainingData_y = y_train.take(train_index,axis=0)
    trainingData_X = X_train.take(train_index,axis=0)
    validationData_y = y_train.take(val_index,axis=0)
    validationData_X = X_train.take(val_index,axis=0)

    training_data = tf.data.Dataset.from_tensor_slices((trainingData_X, trainingData_y))
    training_data = training_data.batch(32)

    validation_data = tf.data.Dataset.from_tensor_slices((validationData_X, validationData_y))
    validation_data = validation_data.batch(32)

    results = cnn(hyper[0],training_data,validation_data)

    VALIDATION_ACCURACY.append(results['binary_accuracy'])
    VALIDATION_LOSS.append(results['loss'])

  accuracyMean = np.sum(VALIDATION_ACCURACY) / len(VALIDATION_ACCURACY)
  lossMean = np.sum(VALIDATION_LOSS) / len(VALIDATION_LOSS)

  print("mean loss: ",lossMean )
  print("mean accuracy: ",accuracyMean)

  return [lossMean], [hyperToString(hyper)]

## Training
### Model Architecture

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import datetime, os

def XceptCnn(drop,train,val):

  base_model = Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=False,
  )
  base_model.trainable = False
  inputs = Input(shape=(299, 299, 3))

  norm_layer = keras.layers.experimental.preprocessing.Normalization()
  mean = np.array([127.5] * 3)
  var = mean ** 2

  outputs = norm_layer(inputs)
  norm_layer.set_weights([mean, var])


  outputs = base_model(outputs,training=False)
  outputs = GlobalAveragePooling2D()(outputs)
  outputs = Dense(1024, activation='relu')(outputs)
  outputs = Dropout(drop)(outputs)
  outputs = Dense(512, activation='relu')(outputs)
  outputs = Dropout(drop)(outputs)
  outputs = Dense(2, activation='relu')(outputs)
  outputs = Dense(1, activation='sigmoid')(outputs)

  model = Model(inputs,outputs)

  #model.summary()


  # logdir = os.path.join("saved_models/models","Drop"+str(drop)+"_learn"+str(learn)+"_epoch"+str(epoch)+".h5")
  # tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
  # checkpoint = ModelCheckpoint(logdir, 
    #                         monitor='val_loss', verbose=0, 
    #                         save_best_only=True, mode='min')
  # callbackList = [checkpoint]

  model.compile(
      optimizer=keras.optimizers.Adam(),
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
      metrics=[keras.metrics.BinaryAccuracy()],
  )

  epochs = 30 #int(epoch)
  model.fit(train, epochs=epochs, validation_data=val)
  # model.fit(train, epochs=epochs, validation_data=val,callbacks=callbackList)

  # model.load_weights(logdir)
  results = model.evaluate(val)
  results = dict(zip(model.metrics_names,results))
  tf.keras.backend.clear_session()
  return results

In [ ]:
def loops(cnn, hyper_parameters, variable_arguments, steps, index,values,labels):
  if(len(variable_arguments) == 0):
    return KFcrossVal(cnn,hyper_parameters,train_y,train_X)
  
  hyper_parameters = np.concatenate((hyper_parameters,[0]))
  
  for variable in np.arange(variable_arguments[0][0], variable_arguments[0][1] + steps[index], steps[index]):
    hyper_parameters[index] = variable
    result_value, result_label = loops(cnn,hyper_parameters,variable_arguments[1:],steps,index + 1,values,labels)
    print("results-----------------")
    print(result_value, result_label)
    values = np.concatenate((values, result_value))
    labels = np.concatenate((labels, result_label))
  
  return values, labels

# def XceptCnn(drop, learn, epoch, train, val):
values = []
labels = []
hyper_parameters_range = [[0.48,0.49]]
steps =                  [0.01]
valores, etiquetas = loops(XceptCnn,[],hyper_parameters_range,steps,0,values,labels)

------- model_P0-0.48
Epoch 1/30
42/42 [==============================] - 12s 276ms/step - loss: 0.4995 - binary_accuracy: 0.7526 - val_loss: 0.3495 - val_binary_accuracy: 0.8322
Epoch 2/30
42/42 [==============================] - 10s 242ms/step - loss: 0.3659 - binary_accuracy: 0.8748 - val_loss: 0.3004 - val_binary_accuracy: 0.8993
Epoch 3/30
42/42 [==============================] - 11s 253ms/step - loss: 0.3207 - binary_accuracy: 0.8936 - val_loss: 0.2820 - val_binary_accuracy: 0.9463
Epoch 4/30
42/42 [==============================] - 11s 255ms/step - loss: 0.2976 - binary_accuracy: 0.9108 - val_loss: 0.2466 - val_binary_accuracy: 0.9530
Epoch 5/30
42/42 [==============================] - 11s 265ms/step - loss: 0.2603 - binary_accuracy: 0.9348 - val_loss: 0.2512 - val_binary_accuracy: 0.9463
Epoch 6/30
42/42 [==============================] - 10s 248ms/step - loss: 0.2570 - binary_accuracy: 0.9378 - val_loss: 0.2912 - val_binary_accuracy: 0.8993
Epoch 7/30
42/42 [==================

42/42 [==============================] - 10s 249ms/step - loss: 0.6164 - binary_accuracy: 0.7024 - val_loss: 0.6145 - val_binary_accuracy: 0.7047
Epoch 23/30
42/42 [==============================] - 10s 244ms/step - loss: 0.6155 - binary_accuracy: 0.7024 - val_loss: 0.6136 - val_binary_accuracy: 0.7047
Epoch 24/30
42/42 [==============================] - 10s 246ms/step - loss: 0.6147 - binary_accuracy: 0.7024 - val_loss: 0.6128 - val_binary_accuracy: 0.7047
Epoch 25/30
42/42 [==============================] - 11s 251ms/step - loss: 0.6139 - binary_accuracy: 0.7024 - val_loss: 0.6121 - val_binary_accuracy: 0.7047
Epoch 26/30
42/42 [==============================] - 10s 243ms/step - loss: 0.6133 - binary_accuracy: 0.7024 - val_loss: 0.6114 - val_binary_accuracy: 0.7047
Epoch 27/30
42/42 [==============================] - 10s 243ms/step - loss: 0.6127 - binary_accuracy: 0.7024 - val_loss: 0.6109 - val_binary_accuracy: 0.7047
Epoch 28/30
42/42 [==============================] - 10s 245ms/s

Epoch 13/30
42/42 [==============================] - 10s 242ms/step - loss: 0.2265 - binary_accuracy: 0.9461 - val_loss: 0.1752 - val_binary_accuracy: 0.9797
Epoch 14/30
42/42 [==============================] - 10s 243ms/step - loss: 0.2729 - binary_accuracy: 0.9101 - val_loss: 0.1877 - val_binary_accuracy: 0.9662
Epoch 15/30
42/42 [==============================] - 10s 242ms/step - loss: 0.2111 - binary_accuracy: 0.9476 - val_loss: 0.1999 - val_binary_accuracy: 0.9662
Epoch 16/30
42/42 [==============================] - 10s 250ms/step - loss: 0.1973 - binary_accuracy: 0.9551 - val_loss: 0.1641 - val_binary_accuracy: 0.9797
Epoch 17/30
42/42 [==============================] - 10s 248ms/step - loss: 0.1876 - binary_accuracy: 0.9640 - val_loss: 0.1911 - val_binary_accuracy: 0.9595
Epoch 18/30
42/42 [==============================] - 10s 241ms/step - loss: 0.1884 - binary_accuracy: 0.9640 - val_loss: 0.1581 - val_binary_accuracy: 0.9797
Epoch 19/30
42/42 [==============================] -

Epoch 4/30
42/42 [==============================] - 10s 246ms/step - loss: 0.2744 - binary_accuracy: 0.8869 - val_loss: 0.1976 - val_binary_accuracy: 0.9595
Epoch 5/30
42/42 [==============================] - 11s 253ms/step - loss: 0.2094 - binary_accuracy: 0.9139 - val_loss: 0.1098 - val_binary_accuracy: 0.9662
Epoch 6/30
42/42 [==============================] - 10s 247ms/step - loss: 0.1816 - binary_accuracy: 0.9154 - val_loss: 0.1128 - val_binary_accuracy: 0.9797
Epoch 7/30
42/42 [==============================] - 10s 247ms/step - loss: 0.1298 - binary_accuracy: 0.9461 - val_loss: 0.0672 - val_binary_accuracy: 0.9662
Epoch 8/30
42/42 [==============================] - 11s 251ms/step - loss: 0.1287 - binary_accuracy: 0.9513 - val_loss: 0.0590 - val_binary_accuracy: 0.9730
Epoch 9/30
42/42 [==============================] - 10s 245ms/step - loss: 0.0924 - binary_accuracy: 0.9640 - val_loss: 0.0619 - val_binary_accuracy: 0.9797
Epoch 10/30
42/42 [==============================] - 10s 2

42/42 [==============================] - 10s 249ms/step - loss: 0.1808 - binary_accuracy: 0.9551 - val_loss: 0.1891 - val_binary_accuracy: 0.9459
Epoch 26/30
42/42 [==============================] - 10s 242ms/step - loss: 0.1655 - binary_accuracy: 0.9640 - val_loss: 0.1654 - val_binary_accuracy: 0.9527
Epoch 27/30
42/42 [==============================] - 10s 244ms/step - loss: 0.1540 - binary_accuracy: 0.9670 - val_loss: 0.1840 - val_binary_accuracy: 0.9324
Epoch 28/30
42/42 [==============================] - 10s 246ms/step - loss: 0.1454 - binary_accuracy: 0.9685 - val_loss: 0.1826 - val_binary_accuracy: 0.9527
Epoch 29/30
42/42 [==============================] - 10s 248ms/step - loss: 0.1441 - binary_accuracy: 0.9693 - val_loss: 0.2066 - val_binary_accuracy: 0.9459
Epoch 30/30
5/5 [==============================] - 1s 169ms/step - loss: 0.1663 - binary_accuracy: 0.9527
Epoch 1/30
42/42 [==============================] - 11s 265ms/step - loss: 0.6900 - binary_accuracy: 0.6861 - val_lo

In [ ]:
print(valores)
print(etiquetas)
#print(XceptCnn(0.5,0.001,10,train_dataset_chido,val_dataset_chido))

#KFcrossVal(XceptCnn,[0.5,0.01,30],train_y,train_X)
